In [ ]:
from datetime import datetime
import time
import os
from tqdm import tqdm
import pandas as pd


In [2]:
with open('./Type_names.txt') as f:
    projects = f.readlines()
    projects = [i.rstrip() for i in projects]
print('prj nums'.format(len(projects)))
# projects = ['\\AarogyaSetu']


FileNotFoundError: [Errno 2] No such file or directory: './Type_names.txt'

In [5]:
import collections

hashTable = collections.defaultdict(list)
for project in tqdm(projects):


    fileName = []
    clonePairs = []


    creationStr = "//CreationDate = "

    def readSO(fileInfo):
        sp1 = fileInfo.split('.')
        fileIndex = int(sp1[0]) - 1

        file_name = fileName[fileIndex]


        if 'inpluslab' in file_name:
            file_name = file_name.replace('F:\\inpluslab\\wt', 'D:\\CCFinder')

        file_name = file_name.replace('F:\\', 'D:\\')

        SOfile = open(file_name,'r', encoding='UTF-8')
        for line in SOfile:
            if creationStr in line:
                return line[17:27], file_name+'#'+sp1[1]

        return "2008-01-01", file_name+'#'+sp1[1]

    def readCommit(fileInfo):
        sp1 = fileInfo.split('.')
        fileIndex = int(sp1[0]) - 1
        fileName[fileIndex] = fileName[fileIndex].replace('F:\\', 'D:\\')
        SOfile = open(fileName[fileIndex],'r', encoding='UTF-8')
        line = SOfile.readline()
        line1 = SOfile.readline()
        line1 = SOfile.readline()
        SOfile.close()
        pretime = line[12:]
        time = datetime.strptime(pretime, '%a %b %d %H:%M:%S CST %Y\n').strftime('%Y-%m-%d')

        return time,fileName[fileIndex]

    file = open(r"D:\CCFinder\NewAns" + '\\' + project + ".txt")
    # allCommit = [0 for i in range(3)] #add delete update other
    while 1:
        line = file.readline()
        if not line:
            break

        if line == 'source_files {\n':
            while 1:
                fileLine = file.readline()
                if fileLine == '}\n':
                    break
                listName = fileLine.split('\t')
                fileName.append(listName[1])

        if line == 'clone_pairs {\n':
            while 1:
                PairsLine = file.readline()
                if PairsLine == '}\n':
                    break
                clonePairs.append(PairsLine)
    file.close()

    for i in range(len(clonePairs)//2):

        listPair = clonePairs[i].split('\t')

        time1, SOfile = readSO(listPair[1])
        time2,Commitfile = readCommit(listPair[2])

        if(time1 < time2):

            if hashTable. __contains__(SOfile):
                # hashTable[SOfile] = hashTable[SOfile] + 1
                hashTable[SOfile][0] = hashTable[SOfile][0] + 1
                hashTable[SOfile].append(Commitfile)
            else:
                # hashTable[SOfile] = 1
                hashTable[SOfile].append(1)
                hashTable[SOfile].append(Commitfile)

100%|██████████| 793/793 [08:47<00:00,  1.50it/s]


In [6]:
so_df = pd.DataFrame(columns=['so_file','nums'])
so_df['so_file'] = list(hashTable.keys())
# so_df['nums'] = list(hashTable.values())
temp_series = pd.Series(list(hashTable.values()))
so_df['nums'] = temp_series.apply(lambda x: x[0])
so_df['Commits involved'] = temp_series.apply(lambda x: x[1:])


In [7]:
so_line_idx_list = []

for i in tqdm(range(len(so_df['so_file']))):
    file_info = so_df['so_file'][i].split('#')
    with open('D:/code.ccfxprepdir/'+file_info[0].split('\\')[-1]+'.java.2_0_0_0.default.ccfxprep','r') as f:
        line_list = f.readlines()
        idx = int(file_info[1].split('-')[0])-1
        so_line_idx = int(line_list[idx].split('.')[0], 16)
        so_line_idx_list.append(so_line_idx)

so_df['SO_line_idx'] = so_line_idx_list

100%|██████████| 134302/134302 [19:25<00:00, 115.26it/s]


In [8]:
save_so_info = []
for i in tqdm(range(len(so_df['so_file']))):
    file_info = so_df['so_file'][i].split('#')
    with open(file_info[0],'r',encoding='utf-8') as SOfile:
        SO_line_list = SOfile.readlines()
        so_line_idx = so_df['SO_line_idx'][i]

        flag = 0
        for k in range(so_line_idx-2, -1, -1):
            if '//CreationDate =' in SO_line_list[k]:
                save_so_info.append(SO_line_list[k])
                flag = 1
                break

        if flag==0:
            temp_split = file_info[0].split('_')
            pre_so_file = temp_split[0]+'_'+str(int(temp_split[1].split('.')[0])-1)+'.java'
            with open(pre_so_file, 'r',encoding='utf-8') as preSOfile:
                preSO_line_list = preSOfile.readlines()
                for j in range(len(preSO_line_list)-1, -1,-1):
                    if '//CreationDate =' in preSO_line_list[j]:
                        save_so_info.append(preSO_line_list[j])
                        break


100%|██████████| 134302/134302 [07:11<00:00, 311.39it/s]


In [10]:
so_df['so_info'] = save_so_info

In [62]:
so_df.to_excel('./so and commit corresponding information.xlsx',index=False)

In [1]:
so_df.info()

NameError: name 'so_df' is not defined

In [ ]:
so_df['so_info'][0]

In [61]:
for k,v in tqdm(zip(so_df['so_info'], hashTable.values())):
    commit_list = v[1:]
    for commit_name in commit_list:
        with open('D:/CCFinder/Commit2So/'+commit_name.replace('\\', '-').replace(':','')+'.txt','a',encoding='utf-8') as f:
            f.write(k)



134302it [05:20, 418.45it/s] 


In [13]:
list(hashTable.values())[0]


[2,
 'D:\\CCFinder\\NewData\\APIJSON\\0\\203.java',
 'D:\\CCFinder\\NewData\\APIJSON\\1\\509.java']